# NYC Bus Analysis - Data Collection & Preprocessing

This analysis is based on the Kaggle dataset 'New York City Bus Data' provided by Michael Stone. The purpose of this exercise is to get a better understanding of bus delays in various areas of New York City and attempt to predict bus delays in the future.

This notebook is dedicated to collecting and cleaning data prior to performing analysis. 

In [937]:
#Import statements here
import numpy as np
import pandas as pd
import datetime
from datetime import date
from datetime import timedelta
import geopy

The dataset in Kaggle consists of 4 months' worth of bus data, each provided as a separate file. As the data for each month is fairly large (> 5 million rows), I using one month of data. We'll be using the data for June 2017.

As I am reading the file I am removing any rows with additional fields. The data contains 17 columns. Additional fields were found on a only a few lines, typically on column 18. Any column after column 17 was not labled and it was difficult to decipher what the fields in these columns represented. Hence, rows with additional fields were removed to reduce anomalies in the data.

In [938]:
dataset = pd.read_csv('mta_1706.csv',sep=',',error_bad_lines=False).dropna()

b'Skipping line 53192: expected 17 fields, saw 18\nSkipping line 56712: expected 17 fields, saw 18\nSkipping line 64874: expected 17 fields, saw 18\n'
b'Skipping line 80636: expected 17 fields, saw 18\nSkipping line 97072: expected 17 fields, saw 18\n'
b'Skipping line 107697: expected 17 fields, saw 18\nSkipping line 109317: expected 17 fields, saw 18\nSkipping line 119951: expected 17 fields, saw 18\n'
b'Skipping line 134667: expected 17 fields, saw 18\nSkipping line 139923: expected 17 fields, saw 18\nSkipping line 144309: expected 17 fields, saw 18\nSkipping line 157927: expected 17 fields, saw 18\nSkipping line 158907: expected 17 fields, saw 18\nSkipping line 161424: expected 17 fields, saw 18\n'
b'Skipping line 173221: expected 17 fields, saw 18\nSkipping line 173623: expected 17 fields, saw 18\nSkipping line 177253: expected 17 fields, saw 18\nSkipping line 181883: expected 17 fields, saw 18\nSkipping line 186684: expected 17 fields, saw 18\nSkipping line 191267: expected 17 fie

b'Skipping line 2831078: expected 17 fields, saw 18\nSkipping line 2831884: expected 17 fields, saw 18\nSkipping line 2832778: expected 17 fields, saw 18\n'
b'Skipping line 2935466: expected 17 fields, saw 18\nSkipping line 2937968: expected 17 fields, saw 18\n'
b'Skipping line 2952504: expected 17 fields, saw 18\n'
b'Skipping line 2982958: expected 17 fields, saw 18\nSkipping line 2991335: expected 17 fields, saw 18\nSkipping line 3006710: expected 17 fields, saw 18\nSkipping line 3013361: expected 17 fields, saw 18\n'
b'Skipping line 3019557: expected 17 fields, saw 18\nSkipping line 3032671: expected 17 fields, saw 18\n'
b'Skipping line 3068454: expected 17 fields, saw 18\nSkipping line 3079236: expected 17 fields, saw 18\n'
b'Skipping line 3081942: expected 17 fields, saw 18\nSkipping line 3095240: expected 17 fields, saw 18\n'
b'Skipping line 3166450: expected 17 fields, saw 18\nSkipping line 3178126: expected 17 fields, saw 18\n'
b'Skipping line 3192452: expected 17 fields, saw 1

b'Skipping line 5158386: expected 17 fields, saw 18\n'
b'Skipping line 5278636: expected 17 fields, saw 18\n'
b'Skipping line 5389023: expected 17 fields, saw 18\nSkipping line 5393062: expected 17 fields, saw 18\n'
b'Skipping line 5410409: expected 17 fields, saw 18\n'
b'Skipping line 5444054: expected 17 fields, saw 18\n'
b'Skipping line 5496628: expected 17 fields, saw 18\nSkipping line 5496629: expected 17 fields, saw 18\n'
b'Skipping line 5510776: expected 17 fields, saw 18\n'
b'Skipping line 5548626: expected 17 fields, saw 18\nSkipping line 5551773: expected 17 fields, saw 18\nSkipping line 5556312: expected 17 fields, saw 18\nSkipping line 5570453: expected 17 fields, saw 18\n'
b'Skipping line 5577387: expected 17 fields, saw 18\nSkipping line 5580723: expected 17 fields, saw 18\nSkipping line 5582075: expected 17 fields, saw 18\nSkipping line 5596334: expected 17 fields, saw 18\nSkipping line 5602325: expected 17 fields, saw 18\n'
b'Skipping line 5606005: expected 17 fields, s

The first 5 lines of the dataset are shown below. For each bus on a particular day and time, the origin and destination of the bus route is provided in latitude and longitude. The next bus stop and the distance of the bus from the next bus stop are provided. In addition, expected arrival time and scheduled arrival time of that bus to that bus stop is provided. 

In [939]:
dataset.head(5)

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime
0,2017-06-01 00:03:34,0,B8,4 AV/95 ST,40.616104,-74.031143,BROWNSVILLE ROCKAWAY AV,40.656048,-73.907379,NYCT_430,40.635170,-73.960803,FOSTER AV/E 18 ST,approaching,76.0,2017-06-01 00:03:59,24:06:14
1,2017-06-01 00:03:43,1,S61,ST GEORGE FERRY/S61 & S91,40.643169,-74.073494,S I MALL YUKON AV,40.575935,-74.167686,NYCT_8263,40.590802,-74.158340,MERRYMOUNT ST/TRAVIS AV,approaching,62.0,2017-06-01 00:03:56,23:58:02
2,2017-06-01 00:03:49,0,Bx10,E 206 ST/BAINBRIDGE AV,40.875008,-73.880142,RIVERDALE 263 ST,40.912376,-73.902534,NYCT_4223,40.886010,-73.912647,HENRY HUDSON PKY E/W 235 ST,at stop,5.0,2017-06-01 00:03:56,24:00:53
3,2017-06-01 00:03:31,0,Q5,TEARDROP/LAYOVER,40.701748,-73.802399,ROSEDALE LIRR STA via MERRICK,40.666012,-73.735939,NYCT_8422,40.668002,-73.729348,HOOK CREEK BL/SUNRISE HY,< 1 stop away,267.0,2017-06-01 00:04:03,24:03:00
4,2017-06-01 00:03:22,1,Bx1,RIVERDALE AV/W 231 ST,40.881187,-73.909340,MOTT HAVEN 136 ST via CONCOURSE,40.809654,-73.928360,NYCT_4710,40.868134,-73.893032,GRAND CONCOURSE/E 196 ST,at stop,11.0,2017-06-01 00:03:56,23:59:38


### Step 1: Consistent Date Time Format

As the dataset contains the fields 'ExpectedArrivalTime' and 'ScheduledArrivalTime' it is possible and useful for our analysis to calculate how delayed a particular bus is. In order to do that, the fields 'ExpectedArrivalTime' and 'ScheduledArrivalTime' need to have a consistent format. We need to remove date information from 'ExpectedArrivalTime' and convert the field to datetime format to perform calculations.

In [940]:
dataset['ExpectedArrivalHour'] = pd.to_datetime(dataset['ExpectedArrivalTime'], format='%Y-%m-%d %H:%M:%S').dt.time

### Step 2: Cleaning Time Values >24 Hours

Another inconsistency between 'ExpectedArrivalTime' and 'ScheduledArrivalTime' is that 'ScheduledArrivalTime' denotes time in hour zero as '24:00:00' while 'ExpectedArrivalTime' denotes time in hour zero as '00:00:00'. This needs to be standardized to accurately calculate bus delay. In the case, we will use '00:00:00' for hour zero. The function 'update_time' performs this conversion and the function is applied 'ScheduledArrivalTime'.

In addition, while running the original function 'update_time', it was noticed that there were'ScheduledArrivalTime' values starting with '25:00:00', '26:00:00','27:00:00' and '28:00:00'. It was difficult to determine whether '25:00:00' was 1 am or that entry was a typo and it was supposed to be '24:00:00'. This was also the case for values with '26:00:00','27:00:00' and '28:00:00'. Since we want to calculate delay accurately, these rows were converted to 'NaN', a string value for 'None'. These rows are removed in the subsequent steps.

In [941]:
def update_times(s):
    if s[:2] == '24':
        s = '00' + s[2:]
    elif s[:2] == '25': 
        s = 'NaN'
    elif s[:2] == '26':
        s = 'NaN'
    elif s[:2] == '27':
        s = 'NaN'
    elif s[:2] == '28':
        s = 'NaN'
    return s

The update_time function is applied to the all the rows in the 'ScheduledArrivalTime' field.

In [942]:
dataset['ScheduledArrivalTime'] = dataset['ScheduledArrivalTime'].apply(update_times)

ScheduledArrivalTime' values starting with '25:00:00', '26:00:00','27:00:00' and '28:00:00' that were converted to 'NaN' are now removed.

In [943]:
dataset = dataset[~dataset['ScheduledArrivalTime'].isin(['NaN','NaT'])]

We are converting'ScheduledArrivalTime' to datetime format as we did with 'ExpectedArrivalTime' above. This helps us perform calculations on both fields.

In [946]:
dataset['ScheduledArrivalTime'] = pd.to_datetime(dataset['ScheduledArrivalTime'], format='%H:%M:%S').dt.time

### Step 3: Function to Calculate Time Difference

A function to calculate time difference is written. The expected arrival time is the actual time the bus is going to arrive at a particular bus stop based on its current location. Scheduled arrival time is the stipulated time that a particular bus is supposed to arrive at a bus stop based on the bus schedule.

The time difference function takes into account multiple scenarios. Importantly, it considers the possibility that a bus that was supposed to arrive before midnight (based on scheduled arrival time) arrives the next day, a few minutes after midnight (based on expected arrival time). Similarly, it also takes into account the possibility that a bus that was supposed to arrive a few minutes after midnight on the next day (based on scheduled arrival time), arrives early a few minutes before midnight on the previous day (based on expected arrival time). The above mentioned two scenarios are based on the assumption that the difference between expected arrival time and scheduled arrival time is no more than twelve hours. In other words, a bus can't be more than 12 hours late or 12 hours early. 

The output of this function is the time difference. If the time difference is a positive number, it means that the bus arrived early. If the time difference is a negative number, it means that the bus arrived late.

In [947]:
def get_time_difference(scheduled, expected):
    #print(scheduled)
    #print(expected)
    if (scheduled.hour> expected.hour):
        if(scheduled.hour - expected.hour >= 12):
            scheduled = datetime.datetime.combine(date.today()-timedelta(1),scheduled)
        else:
            scheduled = datetime.datetime.combine(date.today(), scheduled)
        expected = datetime.datetime.combine(date.today(), expected)

    elif (expected.hour > scheduled.hour):
        if(expected.hour - scheduled.hour >= 12):
            #print("SHOULD NOT BE HERE")
            expected = datetime.datetime.combine(date.today()-timedelta(1), expected)
        else:
            expected = datetime.datetime.combine(date.today(), expected)
        scheduled = datetime.datetime.combine(date.today(), scheduled)

    elif (scheduled.hour == expected.hour):
        scheduled = datetime.datetime.combine(date.today(), scheduled)
        expected= datetime.datetime.combine(date.today(), expected)

    diff = scheduled-expected
    if diff.days < 0:
        #print('negative')
        diff_seconds = diff.seconds - 24*60*60
    else:
        diff_seconds = diff.seconds
    #print(diff_seconds)
    return diff_seconds

In [948]:
dataset['Delay_Seconds'] = np.vectorize(get_time_difference)( dataset['ScheduledArrivalTime'],dataset['ExpectedArrivalHour'] )

Time difference is converted from seconds to minutes to able to interpret the data easily.

In [949]:
dataset['Delay_Minutes'] = dataset['Delay_Seconds']/60

In [950]:
# round delay to 1 dp
dataset['Delay_Minutes'] = dataset['Delay_Minutes'].round() 

On average, buses are delayed and by over 5 minutes.

In [952]:
dataset['Delay_Minutes'].mean()

-5.732088504441816

### Step 4: Data Storage

The 'dataset' dataframe is stored so that can be used in the next notebook 'NYC Bus Analysis - Data Analysis and Visualization'.

In [953]:
%store dataset

Stored 'dataset' (DataFrame)


This is how the dataset looks at the end of pre-processing.

In [954]:
dataset.head(20)

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,ExpectedArrivalHour,Delay_Seconds,Delay_Minutes
0,2017-06-01 00:03:34,0,B8,4 AV/95 ST,40.616104,-74.031143,BROWNSVILLE ROCKAWAY AV,40.656048,-73.907379,NYCT_430,40.635170,-73.960803,FOSTER AV/E 18 ST,approaching,76.0,2017-06-01 00:03:59,00:06:14,00:03:59,135,2.0
1,2017-06-01 00:03:43,1,S61,ST GEORGE FERRY/S61 & S91,40.643169,-74.073494,S I MALL YUKON AV,40.575935,-74.167686,NYCT_8263,40.590802,-74.158340,MERRYMOUNT ST/TRAVIS AV,approaching,62.0,2017-06-01 00:03:56,23:58:02,00:03:56,-354,-6.0
2,2017-06-01 00:03:49,0,Bx10,E 206 ST/BAINBRIDGE AV,40.875008,-73.880142,RIVERDALE 263 ST,40.912376,-73.902534,NYCT_4223,40.886010,-73.912647,HENRY HUDSON PKY E/W 235 ST,at stop,5.0,2017-06-01 00:03:56,00:00:53,00:03:56,-183,-3.0
3,2017-06-01 00:03:31,0,Q5,TEARDROP/LAYOVER,40.701748,-73.802399,ROSEDALE LIRR STA via MERRICK,40.666012,-73.735939,NYCT_8422,40.668002,-73.729348,HOOK CREEK BL/SUNRISE HY,< 1 stop away,267.0,2017-06-01 00:04:03,00:03:00,00:04:03,-63,-1.0
4,2017-06-01 00:03:22,1,Bx1,RIVERDALE AV/W 231 ST,40.881187,-73.909340,MOTT HAVEN 136 ST via CONCOURSE,40.809654,-73.928360,NYCT_4710,40.868134,-73.893032,GRAND CONCOURSE/E 196 ST,at stop,11.0,2017-06-01 00:03:56,23:59:38,00:03:56,-258,-4.0
5,2017-06-01 00:03:40,0,M1,4 AV/E 10 ST,40.731342,-73.990288,HARLEM 147 ST via MADISON,40.821110,-73.935898,NYCT_3831,40.792897,-73.950023,MADISON AV/E 106 ST,approaching,73.0,2017-06-01 00:03:56,00:02:35,00:03:56,-81,-1.0
7,2017-06-01 00:03:29,0,B83,GATEWAY CTR TERM/GATEWAY DR,40.652649,-73.877029,BWAY JCT VN SNDRN AV,40.678139,-73.903572,NYCT_4841,40.648801,-73.882682,PENNSYLVANIA AV/DELMAR LOOP N,< 1 stop away,196.0,2017-06-01 00:04:13,23:58:47,00:04:13,-326,-5.0
8,2017-06-01 00:03:27,0,B82,STILLWELL TERMINAL BUS LOOP,40.577080,-73.981293,SPRING CRK TWRS SEAVIEW AV via KINGS HWY,40.642990,-73.878326,NYCT_6592,40.632258,-73.918318,FLATLANDS AV/RALPH AV,approaching,35.0,2017-06-01 00:03:56,00:00:00,00:03:56,-236,-4.0
9,2017-06-01 00:03:51,1,S59,RICHMOND TER/PARK AV #3,40.640167,-74.130966,HYLAN BL,40.534260,-74.154213,NYCT_8279,40.590689,-74.165811,RICHMOND AV/NOME AV,approaching,31.0,2017-06-01 00:03:56,00:01:14,00:03:56,-162,-3.0
10,2017-06-01 00:03:48,0,Bx28,VALENTINE AV/E 192 ST,40.864079,-73.894615,CO-OP CITY EARHART LA via GUNHILL,40.860828,-73.822670,NYCT_8334,40.876032,-73.829543,DREISER LOOP/DEBS PL,< 1 stop away,207.0,2017-06-01 00:04:29,23:48:35,00:04:29,-954,-16.0


The data is converted to CSV format and saved. This is a backup file that can be used in case any changes are made and we'd like to revert back to the original version.

In [955]:
dataset.to_csv('bus_dataset_clean3.csv')